## Example 14 - Emission footprint of IWT

In [1]:
import opentnsim
print('This notebook has been tested with OpenTNSim version {}'.format(opentnsim.__version__))

This notebook has been tested with OpenTNSim version 1.0.0


In [2]:
# package(s) related to time, space and id
import datetime, time
import platform
import random
import os

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import pyproj
import shapely.geometry
from simplekml import Kml, Style
import folium

# package(s) for data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from osgeo import ogr, osr

# OpenTNSIM
import opentnsim.core as core
import opentnsim.graph_module as graph_module
import opentnsim.plot as plot
import opentnsim.model as model

# Used for mathematical functions
import math             
import json

# Used for making the graph to visualize our problem
import networkx as nx  

# Graph location
location_graph = "Shape-Files/Vaarwegvakken"
name_graph = "Vaarwegvakken.shp"

# Vessel database
location_vessel_database = "Vessels\Vessel-database.csv"

### Initiate Simpy environment

In [3]:
# Start simpy environment
simulation_start = datetime.datetime.now()
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))

### Load and transform graph

**Important**: 

If you use windows and get the following error "ImportError: read_shp requires OGR: http://www.gdal.org/", you probably have [this issue](https://github.com/conda-forge/gdal-feedstock/issues/219). Solving it is possible by running the following commands in your terminal (as explained [here](https://gis.stackexchange.com/questions/294231/installing-gdal-with-anaconda)):

```bash
#Create a new virtual environment
conda create -n testgdal -c conda-forge gdal vs2015_runtime=14

#Activate virtual environment
activate testgdal

#Open Jupyter notebook
jupyer notebook
```

In [4]:
FG = nx.read_shp(os.path.join(location_graph, name_graph), simplify=True)

# The read_shp creates a directed graph with single edges
# We require a directed graph but two-way traffic

FG = FG.to_undirected()
FG = FG.to_directed()

In [5]:
def transform_projection(location_graph, name_graph):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    dataset = driver.Open(os.path.join(location_graph, name_graph))

    # from Layer
    inSpatialRef = dataset.GetLayer().GetSpatialRef()

    # Set up the coordinate reference we want to use, WGS84 - World Geodetic System 1984
    outSpatialRef = osr.SpatialReference()
    outSpatialRef.ImportFromEPSG(4326)

    # Transform the coordinates
    transform = osr.CoordinateTransformation(inSpatialRef, outSpatialRef)
    
    return transform

In [6]:
def change_projection(transform, point):
    point = ogr.CreateGeometryFromWkt(str(point))
    
    point.Transform(transform)
    point.ExportToWkt()
    
    return point.GetX(), point.GetY()

In [7]:
transform = transform_projection(location_graph, name_graph)

FG_new = nx.DiGraph()
nodes_dict = {}

for i, node in enumerate(FG.nodes(data = True)):
    coordinates = change_projection(transform, shapely.geometry.Point(list(FG.nodes)[i][0], list(FG.nodes)[i][1]))
    name = "({:f}, {:f})".format(coordinates[0], coordinates[1])
    geometry = shapely.geometry.Point(coordinates[0], coordinates[1])
    
    nodes_dict[list(FG.nodes)[i]] = name
    FG_new.add_node(name, name = name, Position = coordinates, geometry = geometry, Old = node[1])
    
for edge in FG.edges(data = True):
    node_1 = nodes_dict[edge[0]]
    node_2 = nodes_dict[edge[1]]
    
    VRT_NAAM = edge[2]["VRT_NAAM"]
    VWG_NAAM = edge[2]["VWG_NAAM"]
    BEGKM =  edge[2]["BEGKM"]
    ENDKM =  edge[2]["ENDKM"]
    DIST = np.abs(float(BEGKM) - float(ENDKM))
    
    LINE = (json.loads(edge[2]["Json"])["coordinates"])
    LineString = []
    for coordinates in LINE:
        LineString.append(change_projection(transform, shapely.geometry.Point(coordinates[0], coordinates[1])))
    
    FG_new.add_edge(node_1, node_2, LineString = shapely.geometry.LineString(LineString), 
                    VRT_NAAM = VRT_NAAM, VWG_NAAM = VWG_NAAM, BEGKM = BEGKM, ENDKM = ENDKM, DIST = DIST)

if nx.info(FG) == nx.info(FG_new):
    print("Succes!")

Succes!


### Select only relevant area

In [8]:
# North-East
NE = (4.54, 51.75)
# South-East
SE = (4.54, 51.60)
# South-West
SW = (4.20, 51.60)
# North-West
NW = (4.20, 51.75)

polygon = shapely.geometry.Polygon([NE, SE, SW, NW])

In [9]:
nodes = []
edges = []

for edge in FG_new.edges(data = True):
    node_1 = FG_new.nodes[edge[0]]
    node_2 = FG_new.nodes[edge[1]]
    
    if node_1["geometry"].within(polygon) or node_2["geometry"].within(polygon):
        nodes.append(node_1)
        nodes.append(node_2)
        edges.append(edge)

In [10]:
FG_new = nx.DiGraph ()

for node in nodes:
    FG_new.add_node(node["name"], name = node["name"], Position = node["Position"], geometry = node["geometry"])

for edge in edges:
    FG_new.add_edge(edge[0], edge[1], Info = edge[2])

### Show on map

In [11]:
# Browser
m = folium.Map(location=[51.7, 4.4], zoom_start = 12)

for edge in FG_new.edges(data = True):
#     print(edge[2]["Info"]["VWG_NAAM"])
    points_x = list(edge[2]["Info"]["LineString"].coords.xy[0])
    points_y = list(edge[2]["Info"]["LineString"].coords.xy[1])
    
    line = []
    for i, _ in enumerate(points_x):
        line.append((points_y[i], points_x[i]))
    
    if edge[2]["Info"]["VWG_NAAM"] in ["Voorhavens Jachtensluis", "Voorhavens Volkeraksluizen"]:
        folium.PolyLine(line, color = "red", weight = 5, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)
    
    else:
        folium.PolyLine(line, weight = 2, popup = edge[2]["Info"]["VWG_NAAM"]).add_to(m)

m

### Create locks
We can see on the maps that there are locks on the graph, but that the information on the locks is limited. The following edges represent locks (NB: click the edges to show info:

- Voorhavens Jachtensluis
- Voorhavens Volkeraksluizen

These edges will be replaced by two lock elements. The Jachtensluizen are mainly designed for yachts and pleasure craft, and have a length of 135 meters and a width of 16 meters. The Volkeraksluizen have three chambers each with a length of 330 meters and a width of 24 meters. For now we'll assume that both locks have a maximum allowable draught of 4.5 meters.

Additional information on the locks can be found on the [Rijkswaterstand website (link in Dutch)](https://www.rijkswaterstaat.nl/water/waterbeheer/bescherming-tegen-het-water/waterkeringen/deltawerken/volkeraksluizen.aspx). 

In [12]:
lock_nr_1 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 1", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_2 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 2", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

lock_nr_3 = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Volkerak - 3", 
                        node_1 = "(4.430289, 51.700047)", node_2 = "(4.392555, 51.681251)",
                        lock_length = 330, lock_width = 24, lock_depth = 4.5, 
                        doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

# lock_test = core.IsLock(env = env, nr_resources = 1, priority = True, name = "Jachtensluis", 
#                         node_1 = "(4.395179, 51.691512)", node_2 = "(4.408442, 51.700226)",
#                         lock_length = 330, lock_width = 24, lock_depth = 4.5, 
#                         doors_open = 10 * 60, doors_close = 10 * 60, operating_time = 25 * 60)

In [13]:
for edge in FG_new.edges(data = True):
    if edge[2]["Info"]["VWG_NAAM"] == "Voorhavens Volkeraksluizen":
        # For testing, all locks have the water level at the right side
        lock_nr_1.water_level = "(4.430289, 51.700047)"
        lock_nr_2.water_level = "(4.430289, 51.700047)"
        lock_nr_3.water_level = "(4.430289, 51.700047)"
        
        # Add locks to the correct edge
        FG_new.edges[edge[0], edge[1]]["Lock"] = [lock_nr_1, lock_nr_2, lock_nr_3]

### Load vessel database

In [14]:
# NB: This database needs to be triple checked (!!!) ... ensure tracable sources
# consider use of Vessel database notebook
vessel_db = pd.read_csv(location_vessel_database)
vessel_db.head()

,VesselID,width,length,height_unloaded,height_loaded,draught_unloaded,draught_loaded,emissionfactor,installed_power,own_weight,...,capacity_unloaded,speed_loaded,speed_unloaded,resistance_loaded,resistance_unloaded,is_loaded,avv_class,cemt_class,type,scenario
0,29a24858-4a51-11e9-9792-b469212bff5b,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,80.0,...,0.0,3.083,4.470,16919.0,9188.0,0.0,M1,I,Spits,"['Base Case', 'NS-High', 'NS-Low']"
1,3254257e-4a51-11e9-8548-b469212bff5b,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,80.0,...,0.0,3.083,4.470,16919.0,9188.0,1.0,M1,I,Spits,"['Base Case', 'NS-High', 'NS-Low']"
2,dcc05ff4-4972-11e9-a543-b469212bff5b,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,90.0,...,0.0,3.361,4.638,28770.0,17524.0,0.0,M2,II,Kempenaar,"['Base Case', 'NS-High', 'NS-Low']"
3,dcc05ff4-4972-11e9-a543-b469212bff5b,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,90.0,...,0.0,3.361,4.638,28770.0,17524.0,1.0,M2,II,Kempenaar,"['Base Case', 'NS-High', 'NS-Low']"
4,e6cea50a-4972-11e9-a1f1-b469212bff5b,7.20,70.0,2.0,6.40,1.5,2.7,0.72,120.0,100.0,...,0.0,3.861,4.638,43935.0,22746.0,0.0,M3,III,Hagenaar,"['Base Case', 'NS-High', 'NS-Low']"


In [15]:
# renaming some of the vessel_db fields to align them with the code (NB: this alignment should be improved)
# in the end vessel_db should hold all (and not more than) the fields required by Vessel (see further down) 
vessel_db = vessel_db.rename(columns={'capacity_loaded': 'capacity',
                                      'height_unloaded': 'height_empty', 
                                      'height_loaded': 'height_full', 
                                      'draught_unloaded': 'draught_empty', 
                                      'draught_loaded':  'draught_full',
                                      'resistance_loaded': 'resistance',
                                      'resistance_unloaded': 'resistance_empty',
                                      'avv_class': 'vessel_type'})

# remove parameters that are nog used (later we may want to add these via a mix-in)
del vessel_db['VesselID']
del vessel_db['own_weight']
del vessel_db['capacity_unloaded']
del vessel_db['speed_loaded']
del vessel_db['speed_unloaded']
del vessel_db['is_loaded']
del vessel_db['cemt_class']
del vessel_db['type']
del vessel_db['scenario']

In [16]:
vessel_db.head()

,width,length,height_empty,height_full,draught_empty,draught_full,emissionfactor,installed_power,capacity,resistance,resistance_empty,vessel_type
0,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,260.0,16919.0,9188.0,M1
1,5.05,38.5,2.0,5.25,1.5,2.7,0.73,36.0,260.0,16919.0,9188.0,M1
2,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,422.5,28770.0,17524.0,M2
3,6.06,55.0,2.0,6.10,1.5,2.7,0.72,118.0,422.5,28770.0,17524.0,M2
4,7.20,70.0,2.0,6.40,1.5,2.7,0.72,120.0,520.0,43935.0,22746.0,M3


In [17]:
# Vessel type
Vessel = type('Vessel', 
              (core.Identifiable,      # so we can identify an individual vessel (by name and id)
               core.Movable,           # so we van make the vessel move
               core.HasContainer,      # so that we can indicate the amount of cargo it carries
               core.VesselProperties,  # so that we can provide vessel properties
               core.HasResource,       # so that a vessel be requested to do something
               core.Routeable,         # so that you can provide a route that the vessel will follow
               core.HasEnergy),        # so that you can calculate energy used on route
              {})

In [18]:
generator = model.VesselGenerator(Vessel, vessel_db)

### Run simulation

In [19]:
# NB simpy environment was already initiated 
env.FG = FG_new

In [20]:
# Randomly draw a number of vessels (with random origins and destinations)
vessels = []

# Add 10 vessels to the simulation
for i in range(10):
    random_1 = random.choice(list(env.FG))
    random_2 = random.choice(list(env.FG))
    path = nx.dijkstra_path(env.FG, random_1, random_2)
    
    vessel = generator.generate(env, "Vessel " + str(i))
    vessel.route = path
    vessel.geometry = nx.get_node_attributes(env.FG, "geometry")[vessel.route[0]]
    vessels.append(vessel)
    
    # Add the movements of the vessel to the simulation
    env.process(vessel.move())
#     env.process(start(env, vessel))

In [21]:
# Run simulation
env.run()

### Check results

In [22]:
# pick a vessel from the vessels list and inspect the results (for now it does not seem the locks are working)
for vessel in vessels:
    print(vessel.name)
    df = pd.DataFrame.from_dict(vessel.log)

    # print all messages
    for index, row in df.iterrows():
        print(row['Message'])

Vessel 0
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start
Sailing from node (4.415027, 51.704766) to node (4.423482, 51.710607) start
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start
Sailing from node (4.423482, 51.710607) to node (4.436174, 51.702279) start
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start
Sailing from node (4.436174, 51.702279) to node (4.430289, 51.700047) start
Passing lock start
Passing lock stop
Vessel 1
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) start
Sailing from node (4.383376, 51.716016) to node (4.401283, 51.713412) start
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) start
Sailing from node (4.401283, 51.713412) to node (4.423482, 51.710607) start
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) start
Sailing from node (4.423482, 51.710607) to node (4.415027, 51.704766) start
Sailing from node (4.415027, 51.7

### Calculate energy use

In [23]:
# Define a EnergyCalculation class (this class postprocesses all results and calculates the energy (kWh))
class EnergyCalculation:
    """
    Add information on energy use and effects on energy use.
    """

    def __init__(self, vessel, *args, **kwargs):
        super().__init__(*args, **kwargs)

        """Initialization"""
        self.vessel = vessel
        self.emissionfactor = self.vessel.emissionfactor
        self.energy_use = {"time_start": [], 
                           "time_stop": [], 
                           "edge_start": [],
                           "edge_stop": [],
                           "total_energy": [], 
                           "stationary": []}
        self.co2_footprint = {"total_footprint": 0, "stationary": 0}
        self.mki_footprint = {"total_footprint": 0, "stationary": 0}

    @property
    def power(self):
        # I think this is rougly Eq 2.20 from Vehmeijer (2009), and exactly Eq 15 from Bolt (2003)
        return 2 * (self.vessel.current_speed * self.vessel.resistance * 10 ** -3)  # kW

    def calculate_energy_consumption(self):
        """Calculation of energy consumption based on total time in system and properties"""

        stationary_phase_indicator = [
            "Waiting to enter waiting area stop",
            "Waiting in waiting area stop",
            "Waiting in line-up area stop",
            "Passing lock stop",
        ]
        

        times = self.vessel.log["Timestamp"]
        messages = self.vessel.log["Message"]
        geometries = self.vessel.log["Geometry"]
        for i in range(len(times) - 1):
            delta_t = (times[i + 1] - times[i]).seconds
            if delta_t != 0:
                self.energy_use["time_start"].append(times[i])
                self.energy_use["time_stop"].append(times[i + 1])
                self.energy_use["edge_start"].append(geometries[i])
                self.energy_use["edge_stop"].append(geometries[i + 1])

                if messages[i + 1] in stationary_phase_indicator:
                    # Hier wordt 15% van de energy_delta genomen, voor stationaire fases
                    energy_delta = self.power * delta_t / 3600  # kJ/3600 = kWh
                    self.energy_use["total_energy"].append(energy_delta * 0.15)
                    self.energy_use["stationary"].append(energy_delta * 0.15)

                else:
                    # Hier wordt de energie genomen van varen op kruissnelheid 
                    energy_delta = self.power * delta_t / 3600  # kJ/3600 = kWh
                    self.energy_use["total_energy"].append(energy_delta)
                    self.energy_use["stationary"].append(0)

        # TODO: er moet hier een heel aantal dingen beter worden ingevuld
        # - de kruissnelheid is nu nog per default 1 m/s (zie de Movable mixin). Eigenlijk moet in de 
        #   vessel database ook nog een speed_loaded en een speed_unloaded worden toegevoegd. 
        # - de resistance factor (Rt) moet eigenlijk voor elke vaarweg sectie worden uitgerekend.
        # - er zou nog eens goed gekeken moeten worden wat er gedaan kan worden rond kustwerken 
        #   (nu dus 15% van kruissnelheid, wellicht is er iets mogelijk als wat Vibeke van der Bilt
        #    had gedaan. Die keek naar bronnen waar de energie aan opging: zoveel procent voortstuwing,
        #    zoveel procent boornet, etc.)

In [24]:
# Instantiate an EnergyCalculation object for a particular vessel
# Change nr to look at a different vessel
nr=2
energycalculation = EnergyCalculation(vessels[nr])

# Calculate the energy use based on the vessel log information
energycalculation.calculate_energy_consumption()

# Show the output in a dataframe
df = pd.DataFrame.from_dict(energycalculation.energy_use)
print('{}: Total energy used in this trip is {:.2f} kWh'.format(vessels[nr].name, np.sum(df["total_energy"])))
df

Vessel 2: Total energy used in this trip is 701.52 kWh


,time_start,time_stop,edge_start,edge_stop,total_energy,stationary
0,2020-05-21 23:31:34.000000,2020-05-22 01:37:58.413553,POINT (4.62292558877379 51.62468615267751),POINT (4.517077105310467 51.60722923324587),213.118827,0.00000
1,2020-05-22 01:37:58.413553,2020-05-22 02:24:39.969629,POINT (4.517077105310467 51.60722923324587),POINT (4.482175776949589 51.61995356683045),78.711212,0.00000
2,2020-05-22 02:24:39.969629,2020-05-22 03:17:55.534264,POINT (4.482175776949589 51.61995356683045),POINT (4.436117162846041 51.61825782350502),89.783050,0.00000
3,2020-05-22 03:17:55.534264,2020-05-22 03:34:02.997912,POINT (4.436117162846041 51.61825782350502),POINT (4.427667577930265 51.62518233952007),27.173774,0.00000
4,2020-05-22 03:34:02.997912,2020-05-22 05:02:48.472869,POINT (4.427667577930265 51.62518233952007),POINT (4.370807285850249 51.65742268295148),149.638417,0.00000
5,2020-05-22 05:02:48.472869,2020-05-22 05:06:03.152254,POINT (4.370807285850249 51.65742268295148),POINT (4.368885256028107 51.65870041326348),5.451616,0.00000
6,2020-05-22 05:06:03.152254,2020-05-22 05:17:59.985997,POINT (4.368885256028107 51.65870041326348),POINT (4.362481897892667 51.66376506070044),20.120396,0.00000
7,2020-05-22 05:17:59.985997,2020-05-22 05:58:33.744049,POINT (4.362481897892667 51.66376506070044),POINT (4.388034316357048 51.67880024954439),68.370003,0.00000
8,2020-05-22 05:58:33.744049,2020-05-22 06:05:28.622243,POINT (4.388034316357048 51.67880024954439),POINT (4.392555365264324 51.68125080672722),11.633860,0.00000
9,2020-05-22 06:05:28.622243,2020-05-22 06:50:28.622243,POINT (4.62292558877379 51.62468615267751),POINT (4.430289385790487 51.70004667570937),11.380950,11.38095


### Show loginfo of the locks

In [25]:
pd.DataFrame.from_dict(lock_nr_1.log)

,Message,Timestamp,Value,Geometry
0,Lock doors closing start,2020-05-22 00:15:20.647430,"(4.430289, 51.700047)",0
1,Lock doors closing stop,2020-05-22 00:25:20.647430,"(4.430289, 51.700047)",0
2,Lock chamber converting start,2020-05-22 00:25:20.647430,"(4.430289, 51.700047)",0
3,Lock chamber converting stop,2020-05-22 00:50:20.647430,"(4.392555, 51.681251)",0
4,Lock doors opening start,2020-05-22 00:50:20.647430,"(4.392555, 51.681251)",0
5,Lock doors opening stop,2020-05-22 01:00:20.647430,"(4.392555, 51.681251)",0
6,Lock doors closing start,2020-05-22 06:05:28.622243,"(4.392555, 51.681251)",0
7,Lock doors closing stop,2020-05-22 06:15:28.622243,"(4.392555, 51.681251)",0
8,Lock chamber converting start,2020-05-22 06:15:28.622243,"(4.392555, 51.681251)",0
9,Lock chamber converting stop,2020-05-22 06:40:28.622243,"(4.430289, 51.700047)",0


In [26]:
pd.DataFrame.from_dict(lock_nr_2.log)

,Message,Timestamp,Value,Geometry
0,Lock doors closing start,2020-05-22 00:30:12.580584,"(4.430289, 51.700047)",0
1,Lock doors closing stop,2020-05-22 00:40:12.580584,"(4.430289, 51.700047)",0
2,Lock chamber converting start,2020-05-22 00:40:12.580584,"(4.430289, 51.700047)",0
3,Lock chamber converting stop,2020-05-22 01:05:12.580584,"(4.392555, 51.681251)",0
4,Lock doors opening start,2020-05-22 01:05:12.580584,"(4.392555, 51.681251)",0
5,Lock doors opening stop,2020-05-22 01:15:12.580584,"(4.392555, 51.681251)",0


In [27]:
pd.DataFrame.from_dict(lock_nr_3.log)

,Message,Timestamp,Value,Geometry


### Visualise on Google Earth

In [28]:
plot.vessel_kml(env, vessels)

### Sandbox